In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

# Set env variables for LangSmith (Crucial for observability)
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Enterprise RAG Agent"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
# ... import other necessary modules from LangChain, LangGraph, Chroma, etc.

In [ ]:
# Create an in-memory SQLite database with sample sales data
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float
engine = create_engine('sqlite:///sample_sales.db')
metadata = MetaData()

# Define a sales table
sales_table = Table(
    'sales', metadata,
    Column('id', Integer, primary_key=True),
    Column('product', String),
    Column('region', String),
    Column('amount', Float),
    Column('quarter', Integer)
)
metadata.create_all(engine)

# Insert sample data
with engine.connect() as conn:
    conn.execute(sales_table.insert(), [
        {'product': 'Product A', 'region': 'North', 'amount': 1000.50, 'quarter': 1},
        {'product': 'Product B', 'region': 'South', 'amount': 2500.75, 'quarter': 1},
        {'product': 'Product A', 'region': 'North', 'amount': 1100.00, 'quarter': 2},
        {'product': 'Product C', 'region': 'East', 'amount': 3500.25, 'quarter': 2},
        # ... more sample records
    ])

In [ ]:
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

# Load documents (Using Wikipedia for demo, akin to company docs)
docs = WikipediaLoader(query="Generative artificial intelligence").load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(docs)

# Create and persist the vector database
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OpenAIEmbeddings(),
    persist_directory="./chroma_db"
)
vector_db.persist()

In [ ]:
from langchain.tools.retriever import create_retriever_tool

# Create a retriever from the vector DB
retriever = vector_db.as_retriever(search_kwargs={"k": 3})
rag_tool = create_retriever_tool(
    retriever,
    "search_company_knowledge_base",
    "Searches and returns information from the company's knowledge base about Generative AI and related topics."
)

In [ ]:
from langchain.agents.agent_toolkits import create_sql_agent
from langchain.utilities import SQLDatabase

# Connect to the sample database
db = SQLDatabase(engine=engine)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# This agent itself is a tool that can execute SQL based on natural language
sql_agent = create_sql_agent(
    llm=llm,
    db=db,
    agent_type="openai-tools",
    verbose=True
)

# We need to wrap the agent's execution in a function to make it a LangGraph tool
def sql_query_tool(query):
    """Executes a natural language query on the sales database and returns the results."""
    return sql_agent.invoke({"input": query})

# Note: In a full LangGraph implementation, we would use the `Tool` class to wrap this.

In [ ]:
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.sqlite import SqliteSaver

# Define the agent's state
from typing import TypedDict, Annotated, Sequence
import operator

class AgentState(TypedDict):
    messages: Annotated[Sequence[HumanMessage], operator.add]

# Initialize the LLM
llm = ChatOpenAI(model="gpt-4", temperature=0)

# Create a list of tools
tools = [rag_tool, sql_query_tool]

# Build the graph
graph_builder = StateGraph(AgentState)
agent = create_react_agent(llm, tools)
graph_builder.add_node("agent", agent)
graph_builder.set_entry_point("agent")
graph_builder.add_edge("agent", END)

# Add persistence (important for multi-turn conversations)
memory = SqliteSaver.from_conn_string(":memory:")
app = graph_builder.compile(checkpointer=memory)

In [ ]:
# Example 1: Pure RAG Query
config = {"configurable": {"thread_id": "1"}}
result1 = app.invoke({"messages": [HumanMessage(content="What is a Variational Autoencoder?")]}, config=config)
print(result1['messages'][-1].content)

# Example 2: NL2SQL/BI Query
result2 = app.invoke({"messages": [HumanMessage(content="What were the total sales for Product A in Q1 and Q2? Show me a table.")]}, config)
print(result2['messages'][-1].content)

# Example 3: Complex, Multi-Turn Agentic Query
# The agent should decide to use both tools
result3 = app.invoke({"messages": [HumanMessage(content="Based on our company knowledge, what are some emerging trends in AI? Also, tell me which product has the highest total sales so I can prioritize investing in those trends.")]}, config)
print(result3['messages'][-1].content)

In [ ]:
# Compare OpenAI embeddings with a traditional ML method (e.g., PCA on TF-IDF vectors)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Sample texts
texts = [chunk.page_content[:500] for chunk in chunks[:20]] # first 20 chunks

# Method 1: OpenAI Embeddings (already have)
openai_embeddings = OpenAIEmbeddings().embed_documents(texts)

# Method 2: TF-IDF + PCA
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(texts)
pca = PCA(n_components=2)
tfidf_2d = pca.fit_transform(tfidf_matrix.toarray())

# Plot the traditional ML result
plt.scatter(tfidf_2d[:, 0], tfidf_2d[:, 1])
plt.title("Document Similarity using TF-IDF + PCA (Traditional ML)")
for i, txt in enumerate(range(len(texts))):
    plt.annotate(txt, (tfidf_2d[i, 0], tfidf_2d[i, 1]))
plt.show()